# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Urucará,-2.5364,-57.7600,75.58,97,100,1.19,BR,1612924543
1,Ponta do Sol,32.6667,-17.1000,57.99,71,69,1.99,PT,1612924543
2,Cherskiy,68.7500,161.3000,-2.94,85,0,1.88,RU,1612924388
3,Qaanaaq,77.4840,-69.3632,14.43,79,0,7.02,GL,1612924544
4,Port Alfred,-33.5906,26.8910,68.00,94,2,2.77,ZA,1612924544
...,...,...,...,...,...,...,...,...,...
566,Armidale,-30.5167,151.6500,68.00,70,1,4.00,AU,1612924461
567,Vohibinany,-17.3500,49.0333,72.77,89,100,1.83,MG,1612924637
568,Chifeng,42.2683,118.9636,28.72,89,0,7.16,CN,1612924637
569,Tulu Bolo,8.6667,38.2167,49.21,88,33,4.21,ET,1612924637


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
56,Mozarlândia,-14.7447,-50.5706,71.51,94,0,3.06,BR,1612924552
74,Barra de Tecoanapa,16.5167,-98.7500,76.14,78,0,4.79,MX,1612924554
77,New Norfolk,-42.7826,147.0587,72.00,39,0,1.01,AU,1612924555
95,Minbu,20.1833,94.8833,76.08,34,0,6.46,MM,1612924558
129,Saint-Philippe,-21.3585,55.7679,77.00,65,0,4.61,RE,1612924565
149,Saint-Joseph,-21.3667,55.6167,77.00,65,0,4.61,RE,1612924568
179,Karīmpur,23.9667,88.6167,70.27,35,0,2.35,IN,1612924572
275,Pisco,-13.7000,-76.2167,71.60,78,0,8.05,PE,1612924589
306,Tezu,27.9167,96.1667,75.04,20,0,1.63,IN,1612924594
312,Koutiala,12.3917,-5.4642,72.57,21,0,7.58,ML,1612924399


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
56,Mozarlândia,BR,-14.7447,-50.5706,
74,Barra de Tecoanapa,MX,16.5167,-98.7500,
77,New Norfolk,AU,-42.7826,147.0587,
95,Minbu,MM,20.1833,94.8833,
129,Saint-Philippe,RE,-21.3585,55.7679,
149,Saint-Joseph,RE,-21.3667,55.6167,
179,Karīmpur,IN,23.9667,88.6167,
275,Pisco,PE,-13.7000,-76.2167,
306,Tezu,IN,27.9167,96.1667,
312,Koutiala,ML,12.3917,-5.4642,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 56: Mozarlândia.
Closest hotel in Mozarlândia is Hotel Portugal.
------------
Retrieving Results for Index 74: Barra de Tecoanapa.
Closest hotel in Barra de Tecoanapa is Hotel Garapacho.
------------
Retrieving Results for Index 77: New Norfolk.
Closest hotel in New Norfolk is Tynwald Willow Bend Estate.
------------
Retrieving Results for Index 95: Minbu.
Closest hotel in Minbu is Man Thi Tar Hotel Minbu.
------------
Retrieving Results for Index 129: Saint-Philippe.
Closest hotel in Saint-Philippe is Les Embruns Du Baril.
------------
Retrieving Results for Index 149: Saint-Joseph.
Closest hotel in Saint-Joseph is "Plantation Bed and Breakfast.
------------
Retrieving Results for Index 179: Karīmpur.
Closest hotel in Karīmpur is Hotel Seba.
------------
Retrieving Results for Index 275: Pisco.
Closest hotel in Pisco is HOSTAL MIRAMAR.
------------
Retrieving Results for Index 306: Tezu.
Closest hotel in Tezu is Hotel Shivam.
------------
Retrieving Result

In [9]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
56,Mozarlândia,BR,-14.7447,-50.5706,Hotel Portugal
74,Barra de Tecoanapa,MX,16.5167,-98.7500,Hotel Garapacho
77,New Norfolk,AU,-42.7826,147.0587,Tynwald Willow Bend Estate
95,Minbu,MM,20.1833,94.8833,Man Thi Tar Hotel Minbu
129,Saint-Philippe,RE,-21.3585,55.7679,Les Embruns Du Baril
149,Saint-Joseph,RE,-21.3667,55.6167,"""Plantation Bed and Breakfast"
179,Karīmpur,IN,23.9667,88.6167,Hotel Seba
275,Pisco,PE,-13.7000,-76.2167,HOSTAL MIRAMAR
306,Tezu,IN,27.9167,96.1667,Hotel Shivam
312,Koutiala,ML,12.3917,-5.4642,"Hotel Le Phenix, Mali"


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))